In [2]:
# importing necessary libraries 
import pandas as pd
import os  
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from gensim import corpora, models
import string
from sklearn.feature_extraction.text import TfidfVectorizer
import re
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import ast
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import plotly as plt
import random
import numpy as np
np.random.seed(42)

In [3]:
df_all = pd.read_csv("/project/MERGED.csv")

In [4]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 36 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Brief Name                                          251 non-null    object 
 1   Client                                              251 non-null    object 
 2   Shortlisted Agency                                  251 non-null    object 
 3   Agency Skills                                       210 non-null    object 
 4   Agency Industries                                   210 non-null    object 
 5   Service Description                                 220 non-null    object 
 6   Agency Description                                  226 non-null    object 
 7   Agency services                                     226 non-null    object 
 8   Other Services                                      155 non-null    object 
 9  

In [5]:
columns_to_drop = [
    "Match Score",
    "Lead_George Patten",
    "Lead_Gideon Hyde",
    "Lead_Hannah Fraser",
    "Lead_Kate Walker",
    "Lead_Paul Bowman",
    "Lead_Peter Sayburn",
    "Lead_Phil Kohler",
    "Lead_Robin Scarborough",
    "Number of Agencies Shortlisted Brief", 
    "Client_WA_Interaction"
]

df_all = df_all.drop(columns=columns_to_drop)
df_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 25 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Brief Name                                          251 non-null    object 
 1   Client                                              251 non-null    object 
 2   Shortlisted Agency                                  251 non-null    object 
 3   Agency Skills                                       210 non-null    object 
 4   Agency Industries                                   210 non-null    object 
 5   Service Description                                 220 non-null    object 
 6   Agency Description                                  226 non-null    object 
 7   Agency services                                     226 non-null    object 
 8   Other Services                                      155 non-null    object 
 9  

In [6]:
df = df_all.copy()

# CONTENT BASED FILTERING

## Experiment 1: tfidf vectorization, using cosine similarity

In [7]:
# dropping nans for certain columns
columns_to_check = ['Agency Skills', 'Agency Industries']
df = df.dropna(subset=columns_to_check)

## train test split

In [8]:
from sklearn.model_selection import train_test_split

# Split unique brief names into training and test sets
unique_brief_names = df['Brief Name'].unique()
train_brief_names, test_brief_names = train_test_split(unique_brief_names, 
                                                       test_size=0.2, 
                                                       random_state=42)

# Divide the dataframe into training and test dataframes based on the brief names
train_df = df[df['Brief Name'].isin(train_brief_names)]
test_df = df[df['Brief Name'].isin(test_brief_names)]


In [9]:
from sklearn.model_selection import train_test_split

# Split unique brief names into training and test sets
unique_brief_names = df['Brief Name'].unique()
train_brief_names, test_brief_names = train_test_split(unique_brief_names, 
                                                       test_size=0.2, 
                                                       random_state=42)

# Divide the dataframe into training and test dataframes based on the brief names
train_df = df[df['Brief Name'].isin(train_brief_names)]
test_df = df[df['Brief Name'].isin(test_brief_names)]

# Create a function to concatenate the agency features
def concat_features(x):
    return ' '.join(x['Agency Skills'].astype(str)) + ' ' + \
           ' '.join(x['Agency Industries'].astype(str)) + ' ' + \
           ' '.join(x['Service Description'].astype(str)) + ' ' + \
           ' '.join(x['Agency Description'].astype(str)) + ' ' + \
           ' '.join(x['Agency services'].astype(str)) + ' ' + \
           ' '.join(x['Other Services'].astype(str)) + ' ' + \
           ' '.join(x['Vision'].astype(str)) + ' ' + \
           ' '.join(x['Industry Experience'].astype(str)) + ' ' + \
           ' '.join(x['Agency Locations'].astype(str))

# Group by 'Brief Name' and concatenate the other features
brief_features = df.groupby('Brief Name')[['Brief Region', 
                                           'Sub Brief Taxonomy',
                                           'Main Brief Taxonomy']].apply(
    lambda x: ' '.join(x['Brief Region'].astype(str)) + ' ' +
              ' '.join(x['Sub Brief Taxonomy'].astype(str)) + ' ' +
              ' '.join(x['Main Brief Taxonomy'].astype(str))
).reset_index(name='combined_features')

# Function to process the data
def process_data(dataframe):
    agency_features = dataframe.groupby('Shortlisted Agency').apply(concat_features).reset_index(name='combined_features')
    brief_features = dataframe.groupby('Brief Name')[['Brief Region', 'Sub Brief Taxonomy', 'Main Brief Taxonomy']].apply(
        lambda x: ' '.join(x['Brief Region'].astype(str)) + ' ' +
                  ' '.join(x['Sub Brief Taxonomy'].astype(str)) + ' ' +
                  ' '.join(x['Main Brief Taxonomy'].astype(str))
    ).reset_index(name='combined_features')

    return agency_features, brief_features

## similarity score calculation

In [10]:
# Process the training and test data
train_agency_features, train_brief_features = process_data(train_df)
test_agency_features, test_brief_features = process_data(test_df)

tfidf_vectorizer = TfidfVectorizer(stop_words='english')

# Fit the vectorizer on the training agency features and transform both training and test agency features
agency_matrix_train = tfidf_vectorizer.fit_transform(train_agency_features['combined_features'])
agency_matrix_test = tfidf_vectorizer.transform(test_agency_features['combined_features'])

# Transform the training and test brief features
brief_matrix_train = tfidf_vectorizer.transform(train_brief_features['combined_features'])
brief_matrix_test = tfidf_vectorizer.transform(test_brief_features['combined_features'])

# Calculate similarity matrices for training and test data
similarity_matrix_train = cosine_similarity(brief_matrix_train, agency_matrix_train)
similarity_matrix_test = cosine_similarity(brief_matrix_test, agency_matrix_test)

# recommendation function

In [11]:
def get_recommendations(similarity_matrix, agency_names, top_n=10):
    recommendations = []
    for similarity_scores in similarity_matrix:
        # Get the indexes of the top-N similar agencies
        similar_agencies_idx = similarity_scores.argsort()[::-1][:top_n]
        recommendations.append([agency_names[idx] for idx in similar_agencies_idx])
    return recommendations


agency_names_train = train_agency_features['Shortlisted Agency'].values
agency_names_test = test_agency_features['Shortlisted Agency'].values

recommendations_test = get_recommendations(similarity_matrix_test, agency_names_test)

# Get the actual shortlisted agencies for the test brief names
actual_shortlisted_agencies_test = [test_df[test_df['Brief Name'] == name]['Shortlisted Agency'].tolist() for name in test_brief_features['Brief Name']]


# Evaluation Metrics

In [12]:
#defining a function for evaluation metrics 
# Evaluation Metrics
def evaluate_recommendations(recommendations, actual_shortlisted_agencies):
    total_precision = 0
    for i, recs in enumerate(recommendations):
        true_positives = len(set(recs) & set(actual_shortlisted_agencies[i]))
        precision = true_positives / len(recs)
        total_precision += precision
    return total_precision / len(recommendations)


def calculate_recall(recommendations, actual_shortlisted_agencies):
    total_recall = 0
    for i, recs in enumerate(recommendations):
        true_positives = len(set(recs) & set(actual_shortlisted_agencies[i]))
        recall = true_positives / len(actual_shortlisted_agencies[i])
        total_recall += recall
    return total_recall / len(recommendations)


def calculate_f1_score(precision, recall):
    return 2 * (precision * recall) / (precision + recall)


def calculate_mean_average_precision(recommendations, actual_shortlisted_agencies):
    total_map = 0
    for i, recs in enumerate(recommendations):
        precisions = []
        relevant_count = 0
        for j, rec in enumerate(recs):
            if rec in set(actual_shortlisted_agencies[i]):
                relevant_count += 1
                precisions.append(relevant_count / (j + 1))
        avg_precision = sum(precisions) / len(precisions) if precisions else 0
        total_map += avg_precision
    return total_map / len(recommendations)


def calculate_ndcg(recommendations, actual_shortlisted_agencies, p=10):
    total_ndcg = 0
    for i, recs in enumerate(recommendations):
        dcg = 0
        for j, rec in enumerate(recs[:p]):
            if rec in set(actual_shortlisted_agencies[i]):
                dcg += 1 / np.log2(j + 2)
        idcg = sum([1 / np.log2(i + 2) for i in range(min(len(actual_shortlisted_agencies[i]), p))])
        ndcg = dcg / idcg if idcg > 0 else 0
        total_ndcg += ndcg
    return total_ndcg / len(recommendations)

In [13]:
# Evaluation
precision_test = evaluate_recommendations(recommendations_test, actual_shortlisted_agencies_test)
recall_test = calculate_recall(recommendations_test, actual_shortlisted_agencies_test)
f1_score_test = calculate_f1_score(precision_test, recall_test)
mean_avg_precision_test = calculate_mean_average_precision(recommendations_test, actual_shortlisted_agencies_test)
ndcg_test = calculate_ndcg(recommendations_test, actual_shortlisted_agencies_test)

print("Test Precision:", precision_test)
print("Test Recall:", recall_test)
print("Test F1-Score:", f1_score_test)
print("Test MAP:", mean_avg_precision_test)
print("Test NDCG:", ndcg_test)

Test Precision: 0.11600000000000003
Test Recall: 0.8133333333333335
Test F1-Score: 0.20304160688665715
Test MAP: 0.46115873015873016
Test NDCG: 0.528932667933488


# testing for a brief 

In [14]:
agency_names_test = test_agency_features['Shortlisted Agency'].values

def get_single_recommendation(brief_name, brief_features, similarity_matrix, agency_names, top_n=10):
    # Find the index of the brief name
    brief_idx = brief_features[brief_features['Brief Name'] == brief_name].index[0]
    
    # Get similarity scores and recommended agency indexes
    similarity_scores = similarity_matrix[brief_idx]
    similar_agencies_idx = similarity_scores.argsort()[::-1][:top_n]
    
    # Get the names and scores of the recommended agencies
    recommended_agencies = [agency_names[idx] for idx in similar_agencies_idx]
    recommended_scores = [similarity_scores[idx] for idx in similar_agencies_idx]
    
    return recommended_agencies, recommended_scores

# Example usage
brief_name = test_brief_features['Brief Name'].iloc[0]
recommended_agencies, recommended_scores = get_single_recommendation(brief_name, test_brief_features, similarity_matrix_test, agency_names_test)

print("Recommendations for brief name:", brief_name)
for agency, score in zip(recommended_agencies, recommended_scores):
    print(f"Agency: {agency}, Similarity Score: {score}")


Recommendations for brief name: Awards applications x5
Agency: Five by Five Global, Similarity Score: 0.36258736423024607
Agency: Boost Awards, Similarity Score: 0.19252330615766763
Agency: Innovate Live, Similarity Score: 0.11780251868472916
Agency: Inc, Similarity Score: 0.06942974755056366
Agency: Balmer Agency, Similarity Score: 0.06522322277056924
Agency: In Marketing We Trust, Similarity Score: 0.05182307605955247
Agency: HACKMASTERS, Similarity Score: 0.046888524072648013
Agency: Reed Words, Similarity Score: 0.03773793218185841
Agency: I-AM, Similarity Score: 0.03533520257639661
Agency: xDesign, Similarity Score: 0.03488483246766551


## Experiment 2- Model with encoding for categorical data 

In [8]:
# Create a copy of the DataFrame
df1 = df.copy()

# Data Processing

### multilabel binarization

In [9]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np


# Define the columns that need MultiLabelBinarizer
multilabel_columns = ['Agency Skills', 'Agency Industries', 'Sub Brief Taxonomy',
                      'Main Brief Taxonomy']

# Apply MultiLabelBinarizer to these columns
mlb = MultiLabelBinarizer()
for column in multilabel_columns:
    def process_value(x):
        if isinstance(x, str):
            return tuple(str(val).strip() for val in x.strip('()').split(','))
        else:
            return ()
    
    df1[column] = df[column].apply(process_value)
    mlb_matrix = mlb.fit_transform(df1[column])
    mlb_df = pd.DataFrame(mlb_matrix, columns=[f"{column}_{class_}" for class_ in mlb.classes_])
    df1 = pd.concat([df1, mlb_df], axis=1)
    df1 = df1.drop(column, axis=1)


### one hot encoding for location features

In [10]:
# Display the unique values in 'Agency Locations' and 'Brief Region' columns
print("Unique values in 'Agency Locations':", df1['Agency Locations'].unique())
print("Unique values in 'Brief Region':", df1['Brief Region'].unique())

# 'Agency Locations' column
df1['Agency Locations'] = df1['Agency Locations'].str.split(', ')
df1 = df1.explode('Agency Locations')

# Remove trailing white spaces
df1['Agency Locations'] = df1['Agency Locations'].str.strip()
df1['Brief Region'] = df1['Brief Region'].str.strip()

# Display the unique values after removing white spaces
print("Unique values in 'Agency Locations' after removing white spaces:", df1['Agency Locations'].unique())
print("Unique values in 'Brief Region' after removing white spaces:", df1['Brief Region'].unique())

# One hot encode the exploded DataFrame, prefix the column names, and drop the original column
one_hot_agency = pd.get_dummies(df1['Agency Locations'], prefix='AgencyLocations')
df1 = pd.concat([df1.drop('Agency Locations', axis=1), one_hot_agency], axis=1)

# Repeat the process for 'Brief Region'
one_hot_brief = pd.get_dummies(df1['Brief Region'], prefix='BriefRegion')
df1 = pd.concat([df1.drop('Brief Region', axis=1), one_hot_brief], axis=1)
df1 = df1.reset_index()
df1.head()

Unique values in 'Agency Locations': ['United Kingdom' 'United Kingdom, United States'
 'Australia, United Kingdom, United States' 'Australia' nan
 'Australia, United Kingdom' 'Australia, United States' 'United States']
Unique values in 'Brief Region': ['united kingdom' 'australia' 'united states']
Unique values in 'Agency Locations' after removing white spaces: ['United Kingdom' 'United States' 'Australia' nan]
Unique values in 'Brief Region' after removing white spaces: ['united kingdom' 'australia' 'united states']


,index,Brief Name,Client,Shortlisted Agency,Service Description,Agency Description,Agency services,Other Services,Vision,Industry Experience,...,Main Brief Taxonomy_Spatial_Design),Main Brief Taxonomy_Tech & Development,Main Brief Taxonomy_Tech & Development),Main Brief Taxonomy_User_Experience/User_Interface_Design,AgencyLocations_Australia,AgencyLocations_United Kingdom,AgencyLocations_United States,BriefRegion_australia,BriefRegion_united kingdom,BriefRegion_united states
0,0,Awards applications x5,AXA,Boost Awards,We are the world’s first and largest award ent...,We are the world’s first and largest award ent...,(Public_Relations),(Awards_consultancy),NaN,NaN,...,0,0,0,0,0,1,0,0,1,0
1,1,Awards applications x5,AXA,Reed Words,Verbal branding and copy writing,We make brands and business stronger through l...,"(Marketing_Planning), (Social_&_Content)","(Copywriting), (Naming), (Tone_of_Voice), (Bra...",Making Brands and Businesses Stronger Through ...,"FMCG,Retail,Non-Profit",...,0,0,0,0,0,1,0,0,1,0
2,1,Awards applications x5,AXA,Reed Words,Verbal branding and copy writing,We make brands and business stronger through l...,"(Marketing_Planning), (Social_&_Content)","(Copywriting), (Naming), (Tone_of_Voice), (Bra...",Making Brands and Businesses Stronger Through ...,"FMCG,Retail,Non-Profit",...,0,0,0,0,0,0,1,0,1,0
3,2,Countrywide broker video,AXA,Viewpoint Studios,Advertising and commercial photography and film,Inspired photography and film,"(Creative_&_Production), (Social_&_Content)",(Advertising_and_commercial_photography_and_film),NaN,NaN,...,0,0,0,0,0,1,0,0,1,0
4,3,Countrywide broker video,AXA,Shoreditch Design Studio,The creative agency for your next big thing,The creative agency for your next big thing,"(Strategic_Design), (UX/UI_Design)","(Product_Design), (Print_Design), (Animation),...",We create high quality digital design for busi...,"Automotive,Fitness,Health,Insurance,Fintech,Te...",...,0,0,0,0,1,0,0,0,1,0


# train-test split

In [11]:
# Split unique brief names into training and test sets
unique_brief_names = df1['Brief Name'].unique()
train_brief_names, test_brief_names = train_test_split(unique_brief_names, test_size=0.2, random_state=42)

# Divide the DataFrame into training and test DataFrames based on the brief names
train_df = df1[df1['Brief Name'].isin(train_brief_names)]
test_df = df1[df1['Brief Name'].isin(test_brief_names)]

## a. similarity metric calculation uisng cosine

In [14]:
# Process the data to get agency and brief matrices
def process_data(dataframe):
    numeric_cols = dataframe.select_dtypes(include=['int64', 'float64']).columns.difference(['Brief Name', 'Shortlisted Agency'])
    agency_features = dataframe.groupby('Shortlisted Agency')[numeric_cols].mean()
    brief_features = dataframe.groupby('Brief Name')[numeric_cols].mean()

    return agency_features, brief_features

train_agency_matrix, train_brief_matrix = process_data(train_df)
test_agency_matrix, test_brief_matrix = process_data(test_df)

# Calculate similarity matrices for training and test data
similarity_matrix_train = cosine_similarity(train_brief_matrix, train_agency_matrix)
similarity_matrix_test = cosine_similarity(test_brief_matrix, test_agency_matrix)

agency_names_test = test_agency_matrix.index.values

## recommendation generation function

In [13]:
def get_single_recommendation(brief_name, brief_features, similarity_matrix, agency_names, top_n=10):
    # Find the index of the brief name
    brief_idx = np.where(brief_features.index == brief_name)[0][0]

    # Get similarity scores and recommended agency indexes
    similarity_scores = similarity_matrix[brief_idx]
    similar_agencies_idx = similarity_scores.argsort()[::-1][:top_n]

    # Get the names and scores of the recommended agencies
    recommended_agencies = [agency_names[idx] for idx in similar_agencies_idx]
    recommended_scores = [similarity_scores[idx] for idx in similar_agencies_idx]

    return recommended_agencies, recommended_scores

# Example usage
brief_name = test_brief_matrix.index[0]  # You can change this to any brief name in the test set
recommended_agencies, recommended_scores = get_single_recommendation(brief_name, test_brief_matrix, similarity_matrix_test, agency_names_test)

print("Recommendations for brief name:", brief_name)
for agency, score in zip(recommended_agencies, recommended_scores):
    print(f"Agency: {agency}, Similarity Score: {score}")

Recommendations for brief name: Acuity Magazine Podcast Series
Agency: Ampel, Similarity Score: 1.0000000000000002
Agency: Quill Peak Consulting, Similarity Score: 0.9995524492154881
Agency: Bridge Studio, Similarity Score: 0.9995477694322612
Agency: NMD+, Similarity Score: 0.9995075933367519
Agency: Solution17, Similarity Score: 0.9994702770296426
Agency: Rip + Tear, Similarity Score: 0.9994501085595292
Agency: IE, Similarity Score: 0.9994242421233783
Agency: Tobias, Similarity Score: 0.9993984235779853
Agency: Dark Blue, Similarity Score: 0.999387326631536
Agency: Hedgehog Lab, Similarity Score: 0.9993669581598749


In [15]:
# recommendation generation for all the briefs in the test set 
def generate_recommendations(brief_features, similarity_matrix, agency_names, top_n=10):
    recommendations = []
    for brief_name in brief_features.index:
        recommended_agencies, _ = get_single_recommendation(brief_name, brief_features, similarity_matrix, agency_names, top_n)
        recommendations.append(recommended_agencies)
    return recommendations

# Generate recommendations for both models
cosine_recommendations = generate_recommendations(test_brief_matrix, similarity_matrix_test, agency_names_test)

# preparing the actual shrotlisted agencies
actual_shortlisted_agencies_test = test_df.groupby('Brief Name')['Shortlisted Agency'].apply(list).tolist()


## evaluation metrics 

In [20]:
# using the pre-defined evaluation function 
# Evaluating cosine recommendations
precision_cosine = evaluate_recommendations(cosine_recommendations, actual_shortlisted_agencies_test)
recall_cosine = calculate_recall(cosine_recommendations, actual_shortlisted_agencies_test)
f1_score_cosine = calculate_f1_score(precision_cosine, recall_cosine)
mean_avg_precision_cosine = calculate_mean_average_precision(cosine_recommendations, actual_shortlisted_agencies_test)
ndcg_cosine = calculate_ndcg(cosine_recommendations, actual_shortlisted_agencies_test)

print("Cosine Recommendations Evaluation:")
print("Precision:", precision_cosine)
print("Recall:", recall_cosine)
print("F1-Score:", f1_score_cosine)
print("MAP:", mean_avg_precision_cosine)
print("NDCG:", ndcg_cosine)
print("\n")

Cosine Recommendations Evaluation:
Precision: 0.1571428571428572
Recall: 0.7147108843537415
F1-Score: 0.2576388792130527
MAP: 0.9466269841269842
NDCG: 0.7684759745738843




## b. similarity metric calculation uisng jaccard similarity

In [15]:
def jaccard_similarity(vector1, vector2):
    intersection = np.logical_and(vector1, vector2).sum()
    union = np.logical_or(vector1, vector2).sum()
    return intersection / float(union)


## similarity calculation

In [16]:
def compute_jaccard_matrix(brief_matrix, agency_matrix):
    num_briefs = brief_matrix.shape[0]
    num_agencies = agency_matrix.shape[0]
    similarity_matrix = np.zeros((num_briefs, num_agencies))
    
    for i in range(num_briefs):
        for j in range(num_agencies):
            similarity_matrix[i][j] = jaccard_similarity(brief_matrix.iloc[i].values, agency_matrix.iloc[j].values)
    
    return similarity_matrix

jaccard_matrix_train = compute_jaccard_matrix(train_brief_matrix, train_agency_matrix)
jaccard_matrix_test = compute_jaccard_matrix(test_brief_matrix, test_agency_matrix)


# recommendation generation function

In [21]:
def get_single_recommendation_jaccard(brief_name, brief_features, similarity_matrix, agency_names, top_n=10):
    brief_idx = np.where(brief_features.index == brief_name)[0][0]
    similarity_scores = similarity_matrix[brief_idx]
    similar_agencies_idx = similarity_scores.argsort()[::-1][:top_n]
    recommended_agencies = [agency_names[idx] for idx in similar_agencies_idx]
    recommended_scores = [similarity_scores[idx] for idx in similar_agencies_idx]
    return recommended_agencies, recommended_scores

# Example usage
brief_name = test_brief_matrix.index[0]
recommended_agencies, recommended_scores = get_single_recommendation_jaccard(brief_name, test_brief_matrix, jaccard_matrix_test, agency_names_test)
print("Jaccard Recommendations for brief name:", brief_name)
for agency, score in zip(recommended_agencies, recommended_scores):
    print(f"Agency: {agency}, Similarity Score: {score}")


Jaccard Recommendations for brief name: Acuity Magazine Podcast Series
Agency: Ampel, Similarity Score: 1.0
Agency: CreateFuture, Similarity Score: 0.38596491228070173
Agency: Tobias, Similarity Score: 0.2698412698412698
Agency: Salamandra.uk Animation Studios, Similarity Score: 0.2631578947368421
Agency: off brand., Similarity Score: 0.26229508196721313
Agency: Path Ventures, Similarity Score: 0.2542372881355932
Agency: Blue Feathers, Similarity Score: 0.24193548387096775
Agency: Akcelo, Similarity Score: 0.23333333333333334
Agency: HeyBigMan!, Similarity Score: 0.22413793103448276
Agency: OHMY, Similarity Score: 0.2222222222222222


In [23]:
# recommendation generation for all the briefs in the test set 
def generate_recommendations(brief_features, similarity_matrix, agency_names, top_n=10):
    recommendations = []
    for brief_name in brief_features.index:
        recommended_agencies, _ = get_single_recommendation_jaccard(brief_name, brief_features, similarity_matrix, agency_names, top_n)
        recommendations.append(recommended_agencies)
    return recommendations

# Generate recommendations for both models
jaccard_recommendations = generate_recommendations(test_brief_matrix, jaccard_matrix_test, agency_names_test)

# preparing the actual shrotlisted agencies
actual_shortlisted_agencies_test = test_df.groupby('Brief Name')['Shortlisted Agency'].apply(list).tolist()


# evaluation metrics

In [24]:

# Evaluating Jaccard recommendations
precision_jaccard = evaluate_recommendations(jaccard_recommendations, actual_shortlisted_agencies_test)
recall_jaccard = calculate_recall(jaccard_recommendations, actual_shortlisted_agencies_test)
f1_score_jaccard = calculate_f1_score(precision_jaccard, recall_jaccard)
mean_avg_precision_jaccard = calculate_mean_average_precision(jaccard_recommendations, actual_shortlisted_agencies_test)
ndcg_jaccard = calculate_ndcg(jaccard_recommendations, actual_shortlisted_agencies_test)

print("Jaccard Recommendations Evaluation:")
print("Precision:", precision_jaccard)
print("Recall:", recall_jaccard)
print("F1-Score:", f1_score_jaccard)
print("MAP:", mean_avg_precision_jaccard)
print("NDCG:", ndcg_jaccard)


Jaccard Recommendations Evaluation:
Precision: 0.15714285714285717
Recall: 0.7206632653061226
F1-Score: 0.2580229999584839
MAP: 0.971513605442177
NDCG: 0.7802931481229054


## Experiment 3- Model with BERT EMBEDDINGS 

In [26]:
# !pip install transformers
# !pip install torch

## creating embeddings using BERT

In [27]:
from transformers import BertModel, BertTokenizer
import torch

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
        # Convert the tensor to numpy array after taking the mean
        return outputs.last_hidden_state.mean(dim=1).numpy()
    

In [28]:
agency_columns = [
    "Agency Skills", "Agency Industries", "Service Description", 
    "Agency Description", "Agency services", "Other Services", 
    "Vision", "Industry Experience", "Agency Locations"
]

df['Agency_Profile'] = df[agency_columns].fillna('').apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
unique_agency_profiles = df.groupby('Shortlisted Agency')['Agency_Profile'].apply(' '.join).reset_index()

brief_columns = ["Brief Region", "Sub Brief Taxonomy", "Main Brief Taxonomy"]

df['Brief_Profile'] = df[brief_columns].fillna('').apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
unique_brief_profiles = df.groupby('Brief Name')['Brief_Profile'].apply(' '.join).reset_index()

agency_embeddings = [get_embedding(profile) for profile in unique_agency_profiles['Agency_Profile']]
brief_embeddings = [get_embedding(profile) for profile in unique_brief_profiles['Brief_Profile']]

## similarity calculation and train-test splits

In [29]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

# Split unique brief names and corresponding embeddings
unique_brief_names = unique_brief_profiles['Brief Name'].unique()
train_brief_names, test_brief_names = train_test_split(unique_brief_names, test_size=0.2, random_state=42)

# Split unique agency names and corresponding embeddings
unique_agency_names = unique_agency_profiles['Shortlisted Agency'].unique()

# Extract the corresponding embeddings for train and test
train_brief_embeddings = [brief_embeddings[i] for i in range(len(unique_brief_names)) if unique_brief_names[i] in train_brief_names]
test_brief_embeddings = [brief_embeddings[i] for i in range(len(unique_brief_names)) if unique_brief_names[i] in test_brief_names]

# Convert lists of numpy arrays to numpy array for similarity computation
train_brief_embeddings = np.vstack(train_brief_embeddings)
test_brief_embeddings = np.vstack(test_brief_embeddings)
agency_embeddings = np.vstack(agency_embeddings)

# Calculate similarity matrices for training and test data
similarity_matrix_train = cosine_similarity(train_brief_embeddings, agency_embeddings)
similarity_matrix_test = cosine_similarity(test_brief_embeddings, agency_embeddings)

## recommenation generation function

In [30]:
def get_single_recommendation(brief_name, test_brief_names, similarity_matrix, agency_names, top_n=10):
    # Find the index of the brief name
    brief_idx = np.where(test_brief_names == brief_name)[0][0]

    # Get similarity scores and recommended agency indexes
    similarity_scores = similarity_matrix[brief_idx]
    similar_agencies_idx = similarity_scores.argsort()[::-1][:top_n]

    # Get the names and scores of the recommended agencies
    recommended_agencies = [agency_names[idx] for idx in similar_agencies_idx]
    recommended_scores = [similarity_scores[idx] for idx in similar_agencies_idx]

    return recommended_agencies, recommended_scores


In [31]:
brief_name = test_brief_names[0]
recommended_agencies, recommended_scores = get_single_recommendation(brief_name, test_brief_names, similarity_matrix_test, unique_agency_names)

print("Recommendations for brief name:", brief_name)
for agency, score in zip(recommended_agencies, recommended_scores):
    print(f"Agency: {agency}, Similarity Score: {score}")

Recommendations for brief name: MyAviva Engagement
Agency: Neu, Similarity Score: 0.8752783536911011
Agency: Fun Agency Ltd, Similarity Score: 0.8726052045822144
Agency: Dark Blue, Similarity Score: 0.8697119951248169
Agency: Oxara London, Similarity Score: 0.8680103421211243
Agency: Experience, Similarity Score: 0.8678798079490662
Agency: Apadmi, Similarity Score: 0.8658484816551208
Agency: Solace Digital, Similarity Score: 0.8574975728988647
Agency: Blicx, Similarity Score: 0.8560953140258789
Agency: Tinker Taylor, Similarity Score: 0.8523766994476318
Agency: InspoHub Ltd, Similarity Score: 0.8509337902069092


In [32]:
def generate_recommendations(test_brief_names, similarity_matrix, agency_names, top_n=10):
    recommendations = []
    for brief_name in test_brief_names:
        recommended_agencies, _ = get_single_recommendation(brief_name, test_brief_names, similarity_matrix, agency_names, top_n)
        recommendations.append(recommended_agencies)
    return recommendations
actual_shortlisted_agencies_test = df[df['Brief Name'].isin(test_brief_names)].groupby('Brief Name')['Shortlisted Agency'].apply(list).tolist()


## Evaluation Metrics 

In [35]:
# Generate recommendations for test briefs using BERT embeddings
recommendations_test = generate_recommendations(test_brief_names, similarity_matrix_test, unique_agency_names)

# Compute the evaluation metrics using the functions you provided
precision_emb = evaluate_recommendations(recommendations_test, actual_shortlisted_agencies_test)
recall_emb = calculate_recall(recommendations_test, actual_shortlisted_agencies_test)
f1_score_emb = calculate_f1_score(precision_emb, recall_emb)
mean_avg_precision_emb = calculate_mean_average_precision(recommendations_test, actual_shortlisted_agencies_test)
ndcg_emb = calculate_ndcg(recommendations_test, actual_shortlisted_agencies_test)

print("Test Precision:", precision_emb)
print("Test Recall:", recall_emb)
print("Test F1-Score:", f1_score_emb)
print("Test MAP:", mean_avg_precision_emb)
print("Test NDCG:", ndcg_emb)


Test Precision: 0.02142857142857143
Test Recall: 0.09523809523809523
Test F1-Score: 0.034985422740524776
Test MAP: 0.0790107709750567
Test NDCG: 0.06420332910240766
